# Introduction to SQLite

SQLite is a lightweight, serverless, self-contained SQL database engine. It is widely used in applications ranging from small mobile apps to large-scale web applications due to its simplicity and ease of integration.

While SQLite can be used directly through SQL commands, we will use Python's built-in `sqlite3` module to interact with SQLite databases. This allows us to leverage the power of SQL while working within a Python environment.


▶️ Import `pandas`, `numpy`, and `sqlite3`.


In [1]:
import pandas as pd
import numpy as np
import sqlite3

▶️ Run the code below to create a new SQLite database with a table for employees.


In [2]:
# Connect to my_database.db database file
# If the file does not exist, create a new file
conn = sqlite3.connect("my_database.db")

# Create a cursor
# A cursor enables users of a DBMS to traverse through the result set
# without having to retrieve all results at once
c = conn.cursor()

# Drop (delete) employees table if it already exists
c.execute("DROP TABLE IF EXISTS employees")
conn.commit()

# Define a query
# Triple quotes (""") denote multiline strings
create_table_query = """
CREATE TABLE IF NOT EXISTS employees (
  emp_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  department TEXT,
  salary REAL
);
"""

c.execute(create_table_query)
conn.commit()

We can use the `conn` object to execute SQL commands and manage the database. The `cursor` object allows us to execute SQL queries and fetch results.


▶️ Run the code below to insert sample data into the employees table.


In [3]:
c.executescript(
    """
INSERT INTO employees (name, department, salary) VALUES ('Mia', 'HR', 140000.0);
INSERT INTO employees (name, department, salary) VALUES ('Michael', 'Marketing', 200000.0);
INSERT INTO employees (name, department, salary) VALUES ('Sean', 'HR', 250000.0);
INSERT INTO employees (name, department, salary) VALUES ('Dylan', 'Marketing', 205000.0);
"""
)

conn.commit()

| Line              | Function                  | Explanation                       |
| ----------------- | ------------------------- | --------------------------------- |
| `executescript()` | Run multiple SQL commands | Inserts multiple employee records |
| `commit()`        | Save changes              | Writes all changes permanently    |


## Querying Data

We can use pandas to run SQL queries and load the results into a DataFrame for easier analysis. This is easier than using the `cursor` object directly for data retrieval.

▶️ Query all records from the `employees` table and load them into a DataFrame named `df`.

Note that each row (record) corresponds to an employee, and each column corresponds to an attribute of the employee (e.g., name, department, salary).


In [4]:
df = pd.read_sql_query("SELECT * FROM employees;", con=conn)
display(df)

,emp_id,name,department,salary
0,1,Mia,HR,140000.0
1,2,Michael,Marketing,200000.0
2,3,Sean,HR,250000.0
3,4,Dylan,Marketing,205000.0


:::{tip} Which part is the actual query?

The actual SQL query is the string `"SELECT * FROM employees;"` passed to the `pd.read_sql_query()` function.

If you're using a database client, you would typically enter just the SQL query itself without any surrounding code. In this chapter, however, we are using Python to execute the query, so it is embedded within a function call.

:::


▶️ Query the `employees` table for Marketing department employees sorted by salary in descending order.


In [5]:
df = pd.read_sql_query(
    """
SELECT emp_id, name, salary
FROM employees
WHERE department = "Marketing"
ORDER BY salary DESC;
""",
    con=conn,
)
display(df)

,emp_id,name,salary
0,4,Dylan,205000.0
1,2,Michael,200000.0


---

## CRUD Operations

SQLite supports the standard CRUD (Create, Read, Update, Delete) operations. We have already seen how to create tables and read data. Let's look at how to update and delete records.

We have already used the `INSERT` statement to add records. Similarly, you can use the `DELETE` statement to remove records and the `UPDATE` statement to modify existing records.


▶️ Delete employees from the `"HR"` department.


In [6]:
c.execute("DELETE FROM employees WHERE department = 'HR';")
conn.commit()

You can verify the deletion by querying the `employees` table again. You should see that the employees from the HR department have been removed.


In [7]:
df = pd.read_sql_query("SELECT * FROM employees;", con=conn)
display(df)

,emp_id,name,department,salary
0,2,Michael,Marketing,200000.0
1,4,Dylan,Marketing,205000.0


---

## Comparing Pandas and SQL using Airbnb Listings

In this section, we will compare how to perform similar tasks using both pandas and SQL. We will use a dataset of Airbnb listings for this comparison.


▶️ Create a new SQLite database file named `airbnb.db` and create a table named `listings`.


In [8]:
# Connect to airbnb.db database file
# If the file does not exist, create a new file
conn = sqlite3.connect("airbnb.db")

# Create a cursor
# A cursor enables users of a DBMS to traverse through the result set
# without having to retrieve all results at once
c = conn.cursor()

# Drop (delete) listings table if it already exists
c.execute("DROP TABLE IF EXISTS listings")
conn.commit()

# Define a query
# Triple quotes (''') denote multiline strings
create_table_query = """
CREATE TABLE IF NOT EXISTS listings (
    neighbourhood TEXT,
    room_type TEXT,
    bathrooms REAL,
    bedrooms REAL,
    price REAL,
    number_of_reviews INTEGER,
    review_score REAL,
    is_superhost INT
)
"""

c.execute(create_table_query)
conn.commit()

▶️ Read a CSV file containing Airbnb listings into a pandas DataFrame.


In [9]:
df_listings = pd.read_csv(
    "https://github.com/bdi475/datasets/raw/main/chicago-airbnb-listings-small.csv"
)

display(df_listings.head(5))

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
0,Hyde Park,Private room,1.0,1.0,65.0,181,100.0,1
1,South Lawndale,Entire home/apt,1.0,3.0,117.0,395,96.0,1
2,West Town,Entire home/apt,1.0,3.0,70.0,389,93.0,1
3,Uptown,Private room,1.5,1.0,110.0,250,100.0,1
4,Near North Side,Private room,1.0,1.0,75.0,500,94.0,1


▶️ Populate the `listings` table with data from the CSV file.


In [10]:
tables = list(
    pd.read_sql_query('SELECT * FROM sqlite_master WHERE type="table";', con=conn)[
        "tbl_name"
    ]
)
if "listings" in tables:
    c.execute(f"DELETE FROM listings")
    conn.commit()

df_listings.to_sql(name="listings", index=False, con=conn, if_exists="append")

826

---

### Example 1: Select all rows and columns


▶️ Use pandas.


In [ ]:
display(df_listings)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
0,Hyde Park,Private room,1.0,1.0,65.0,181,100.0,1
1,South Lawndale,Entire home/apt,1.0,3.0,117.0,395,96.0,1
2,West Town,Entire home/apt,1.0,3.0,70.0,389,93.0,1
3,Uptown,Private room,1.5,1.0,110.0,250,100.0,1
4,Near North Side,Private room,1.0,1.0,75.0,500,94.0,1
...,...,...,...,...,...,...,...,...
821,Near North Side,Private room,1.0,1.0,97.0,140,91.0,0
822,Edgewater,Entire home/apt,1.0,1.0,104.0,101,92.0,0
823,Lake View,Entire home/apt,1.0,1.0,106.0,132,99.0,1
824,Near North Side,Private room,1.0,1.0,114.0,329,91.0,0


▶️ Use SQLite.


In [ ]:
query_select_all = """
SELECT *
FROM listings
"""

df_result = pd.read_sql_query(query_select_all, con=conn)
display(df_result)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
0,Hyde Park,Private room,1.0,1.0,65.0,181,100.0,1
1,South Lawndale,Entire home/apt,1.0,3.0,117.0,395,96.0,1
2,West Town,Entire home/apt,1.0,3.0,70.0,389,93.0,1
3,Uptown,Private room,1.5,1.0,110.0,250,100.0,1
4,Near North Side,Private room,1.0,1.0,75.0,500,94.0,1
...,...,...,...,...,...,...,...,...
821,Near North Side,Private room,1.0,1.0,97.0,140,91.0,0
822,Edgewater,Entire home/apt,1.0,1.0,104.0,101,92.0,0
823,Lake View,Entire home/apt,1.0,1.0,106.0,132,99.0,1
824,Near North Side,Private room,1.0,1.0,114.0,329,91.0,0


---

### Example 2: Select only three columns


▶️ Use pandas.


In [ ]:
df_listings[["bathrooms", "bedrooms", "price"]]

,bathrooms,bedrooms,price
0,1.0,1.0,65.0
1,1.0,3.0,117.0
2,1.0,3.0,70.0
3,1.5,1.0,110.0
4,1.0,1.0,75.0
...,...,...,...
821,1.0,1.0,97.0
822,1.0,1.0,104.0
823,1.0,1.0,106.0
824,1.0,1.0,114.0


▶️ Use SQLite.


In [ ]:
query_select_three_columns = """
SELECT bathrooms, bedrooms, price
FROM listings
"""

df_result = pd.read_sql_query(query_select_three_columns, con=conn)
display(df_result)

,bathrooms,bedrooms,price
0,1.0,1.0,65.0
1,1.0,3.0,117.0
2,1.0,3.0,70.0
3,1.5,1.0,110.0
4,1.0,1.0,75.0
...,...,...,...
821,1.0,1.0,97.0
822,1.0,1.0,104.0
823,1.0,1.0,106.0
824,1.0,1.0,114.0


---

### Example 3:Select all columns of expensive listings


▶️ Use pandas.


In [ ]:
df_expensive = df_listings[df_listings["price"] > 400]

display(df_expensive)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
85,Uptown,Entire home/apt,2.5,5.0,875.0,124,97.0,0
385,Near North Side,Entire home/apt,1.0,2.0,500.0,117,94.0,1


▶️ Use SQLite.


In [ ]:
query_expensive_listings = """
SELECT *
FROM listings
WHERE price > 400
"""

df_result = pd.read_sql_query(query_expensive_listings, con=conn)
display(df_result)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
0,Uptown,Entire home/apt,2.5,5.0,875.0,124,97.0,0
1,Near North Side,Entire home/apt,1.0,2.0,500.0,117,94.0,1


---

### Example 4: Select all columns of West Town listings


▶️ Use pandas.


In [ ]:
df_west_town = df_listings[df_listings["neighbourhood"] == "West Town"]

display(df_west_town)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
2,West Town,Entire home/apt,1.0,3.0,70.0,389,93.0,1
14,West Town,Entire home/apt,3.0,3.0,168.0,218,97.0,1
15,West Town,Entire home/apt,1.0,2.0,75.0,189,91.0,0
20,West Town,Private room,2.0,1.0,57.0,127,89.0,0
21,West Town,Private room,2.0,1.0,78.0,143,90.0,0
...,...,...,...,...,...,...,...,...
758,West Town,Private room,1.0,1.0,55.0,126,100.0,1
759,West Town,Entire home/apt,1.0,1.0,95.0,125,98.0,1
771,West Town,Entire home/apt,1.0,2.0,68.0,112,93.0,0
798,West Town,Entire home/apt,1.0,2.0,58.0,140,99.0,1


▶️ Use SQLite.


In [ ]:
query_west_town_listings = """
SELECT *
FROM listings
WHERE neighbourhood = 'West Town'
"""

df_result = pd.read_sql_query(query_west_town_listings, con=conn)
display(df_result)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
0,West Town,Entire home/apt,1.0,3.0,70.0,389,93.0,1
1,West Town,Entire home/apt,3.0,3.0,168.0,218,97.0,1
2,West Town,Entire home/apt,1.0,2.0,75.0,189,91.0,0
3,West Town,Private room,2.0,1.0,57.0,127,89.0,0
4,West Town,Private room,2.0,1.0,78.0,143,90.0,0
...,...,...,...,...,...,...,...,...
133,West Town,Private room,1.0,1.0,55.0,126,100.0,1
134,West Town,Entire home/apt,1.0,1.0,95.0,125,98.0,1
135,West Town,Entire home/apt,1.0,2.0,68.0,112,93.0,0
136,West Town,Entire home/apt,1.0,2.0,58.0,140,99.0,1


---

### Example 5: Sort all listings by price


▶️ Use pandas.


In [ ]:
df_sorted_by_price = df_listings.sort_values(by="price")

display(df_sorted_by_price)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
158,West Ridge,Shared room,2.0,1.0,13.0,153,95.0,0
713,North Lawndale,Shared room,1.0,1.0,14.0,118,91.0,0
108,West Ridge,Shared room,2.0,1.0,16.0,124,95.0,0
50,West Ridge,Shared room,2.0,1.0,16.0,158,92.0,0
792,Lower West Side,Private room,2.0,1.0,19.0,112,94.0,1
...,...,...,...,...,...,...,...,...
123,Near North Side,Entire home/apt,2.0,2.0,344.0,128,94.0,1
250,West Town,Entire home/apt,3.0,3.0,379.0,210,98.0,0
628,Lake View,Entire home/apt,2.0,5.0,400.0,113,99.0,1
385,Near North Side,Entire home/apt,1.0,2.0,500.0,117,94.0,1


▶️ Use SQLite.


In [ ]:
query_sort_by_price = """
SELECT *
FROM listings
ORDER BY price
"""

df_result = pd.read_sql_query(query_sort_by_price, con=conn)
display(df_result)

,neighbourhood,room_type,bathrooms,bedrooms,price,number_of_reviews,review_score,is_superhost
0,West Ridge,Shared room,2.0,1.0,13.0,153,95.0,0
1,North Lawndale,Shared room,1.0,1.0,14.0,118,91.0,0
2,West Ridge,Shared room,2.0,1.0,16.0,158,92.0,0
3,West Ridge,Shared room,2.0,1.0,16.0,124,95.0,0
4,Lower West Side,Private room,2.0,1.0,19.0,112,94.0,1
...,...,...,...,...,...,...,...,...
821,Near North Side,Entire home/apt,2.0,2.0,344.0,128,94.0,1
822,West Town,Entire home/apt,3.0,3.0,379.0,210,98.0,0
823,Lake View,Entire home/apt,2.0,5.0,400.0,113,99.0,1
824,Near North Side,Entire home/apt,1.0,2.0,500.0,117,94.0,1


---

### Example 6: Select only the price column and sort in descending order


▶️ Use pandas.


In [ ]:
df_price = df_listings[["price"]].sort_values(by="price", ascending=False)

display(df_price)

,price
85,875.0
385,500.0
628,400.0
250,379.0
123,344.0
...,...
792,19.0
50,16.0
108,16.0
713,14.0


▶️ Use SQLite.


In [ ]:
query_sort_by_price_desc = """
SELECT price
FROM listings
ORDER BY price DESC
"""

df_result = pd.read_sql_query(query_sort_by_price_desc, con=conn)
display(df_result)

,price
0,875.0
1,500.0
2,400.0
3,379.0
4,344.0
...,...
821,19.0
822,16.0
823,16.0
824,14.0


---

### Example 7: A complicated query


▶️ Use pandas.


In [ ]:
df_complicated = df_listings[
    (df_listings["price"] < 100) & (df_listings["number_of_reviews"] > 500)
].sort_values(by="number_of_reviews", ascending=False)
df_complicated = df_complicated[
    ["room_type", "bathrooms", "bedrooms", "price", "number_of_reviews"]
]

display(df_complicated)

,room_type,bathrooms,bedrooms,price,number_of_reviews
262,Private room,1.0,1.0,80.0,658
16,Entire home/apt,1.0,1.0,39.0,642
26,Entire home/apt,1.0,1.0,85.0,625
784,Hotel room,1.0,1.0,74.0,607
43,Entire home/apt,1.0,1.0,75.0,543
170,Entire home/apt,1.0,2.0,77.0,515
44,Entire home/apt,1.0,1.0,65.0,504


▶️ Use SQLite.


In [ ]:
query_final = """
SELECT room_type, bathrooms, bedrooms, price, number_of_reviews
FROM listings
WHERE (price < 100) AND (number_of_reviews > 500)
ORDER BY number_of_reviews DESC
"""

df_result = pd.read_sql_query(query_final, con=conn)
display(df_result)

,room_type,bathrooms,bedrooms,price,number_of_reviews
0,Private room,1.0,1.0,80.0,658
1,Entire home/apt,1.0,1.0,39.0,642
2,Entire home/apt,1.0,1.0,85.0,625
3,Hotel room,1.0,1.0,74.0,607
4,Entire home/apt,1.0,1.0,75.0,543
5,Entire home/apt,1.0,2.0,77.0,515
6,Entire home/apt,1.0,1.0,65.0,504
